<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
            Dr. Eric Van Dusen<br>
        Andrei Caprau<br>
        Umar Maniku<br></p></td></tr>
</table>

# Lab 7: Finance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sympy
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from scipy import stats
import warnings
from datascience import *
warnings.filterwarnings('ignore')
plt.style.use("seaborn-muted")
%matplotlib inline
import nbforms
feedback = nbforms.Form("feedback_form_config.json")

## Compounding Interest

We mentioned in this week's lesson how interest in one year builds on top of not only the principal amount of money deposited into an account, but also the interest earned in previous years. This can be very powerful.

Below is a tool that simulates a retirement or savings account. Each year, a fixed amount is deposited into the account. This is represented by `deposits`. In addition, the interest rate the account yields is `r`, and `t` is how many years this account accrues interest.

In [ ]:
def plot_comp_interest(deposits, r, t):
    balance = []
    balance.append(deposits)
    for _ in range(t):
        balance.append(balance[-1] * (1 + r) + deposits)
    plt.figure(figsize=(8,6))
    plt.plot(np.arange(t + 1), balance)
    plt.xlabel('Time in Years')
    plt.ylabel('Account Balance')
    plt.title('Account Balance Over Time')
    plt.ylim((0, 1000000))
    plt.xlim((0, t))

In [ ]:
deposits_slider = widgets.IntSlider(min = 500, max = 10000, step = 500, value = 5000)
r_slider = widgets.FloatSlider(min = 0.01, max = 0.2, step = 0.01, value = 0.1)
t_slider = widgets.IntSlider(min = 10, max = 30, step = 1, value = 20)
display(widgets.interactive(plot_comp_interest, deposits = deposits_slider, r = r_slider, t = t_slider))

Try out the sliders.
1. What happens as you increase the amount of fixed deposits?
2. What happens as the interest rate increases?
3. What happens as you invest into the account for longer periods of time?
4. Why do people frequently say to start saving for retirement at a young age?

## Black-Scholes

Recall the somewhat complicated-looking formula for the price of a European call and put option. Recall that for European options, we assume that you can only exercise the option on expiration. We also assume that there are no dividends, that stock prices are random and log-normally-distributed, and that the risk-free rate and volatility are constant across time. Do you think these are reasonable assumptions?

Below is code that implements those equations and plots the option price across a range of possible prices of underlying stock. Also included in the plot is the payoff of the option when it's exercised.

In [ ]:
def long_call(S, K, sigma, rf,  time):
    d1 = (np.log(S/K) + (rf + 0.5*(sigma**2)) * time) / (sigma * (np.sqrt(time)))
    d2 = d1 - sigma * np.sqrt(time)
    return S * stats.norm.cdf(d1) - K * np.exp(-rf*time) * stats.norm.cdf(d2)

def long_put(S, K, sigma, rf, time):
    d1 = (np.log(S/K) + (rf + 0.5*(sigma**2)) * time)/(sigma * (np.sqrt(time)))
    d2 = d1 - sigma*np.sqrt(time)
    return K*np.exp(-rf*time)*stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)

def plot(typ, S, K, sigma, rf, time):
    plt.figure(figsize = (8,4))
    
    if typ == "lc":
        payoff = np.maximum(0, np.linspace(1, 81, 500) - K)
        prices = [long_call(i, K, sigma, rf, time) for i in np.linspace(1, 81, 500)]
        plt.xlim(0,70)
        plt.ylim(-10,50)
        plt.title("Black-Scholes Long Call Price")
        
    if typ == "lp":
        payoff = np.maximum(0, K - np.linspace(1, 81, 500))
        prices = [long_put(i, K, sigma, rf, time) for i in np.linspace(1, 81, 500)]
        plt.xlim(0,70)
        plt.ylim(-10,50)
        plt.title("Black-Scholes Long Put Price")
        
    if typ == "sc":
        payoff = -np.maximum(0, np.linspace(1, 81, 500)-K)
        prices = -np.array([long_call(i, K, sigma, rf, time) for i in np.linspace(1, 81, 500)])
        plt.xlim(0,70)
        plt.ylim(-50,10)
        plt.title("Black-Scholes Short Call Price")
        
    if typ == "sp":
        payoff = -np.maximum(0, K-np.linspace(1, 81, 500))
        prices = -np.array([long_put(i, K, sigma, rf, time) for i in np.linspace(1, 81, 500)])
        plt.xlim(0,70)
        plt.ylim(-50,10)
        plt.title("Black-Scholes Short Put Price")
    
    plt.plot(np.linspace(1, 81, 500), payoff)
    plt.plot(np.linspace(1, 81, 500), prices)
    plt.xlabel("Underlying Asset Price")
    plt.legend(["Payoff Diagram", "Option Price"])
    plt.ylabel("Price/Payoff")

### Long Call

In [ ]:
# Long Call
rf_slider = widgets.FloatSlider(min = 0.01, max = 0.2, step = 0.01, value = 0.1)
sigma_slider = widgets.FloatSlider(min = 0.1, max = 0.9, step = 0.1, value = 0.5)
K_slider = widgets.IntSlider(min = 10, max = 60, step = 5, value = 30)
time_slider = widgets.FloatSlider(min = 0.1, max = 2, step = 0.1, value = 1)

display(widgets.interactive(plot, K = K_slider, typ = fixed("lc"), S = fixed(40), 
                            time = time_slider, rf = rf_slider, sigma = sigma_slider))

### Long Put

In [ ]:
# Long Put
rf_slider = widgets.FloatSlider(min = 0.01, max = 0.2, step = 0.01, value = 0.1)
sigma_slider = widgets.FloatSlider(min = 0.1, max = 0.9, step = 0.1, value = 0.5)
K_slider = widgets.IntSlider(min = 10, max = 60, step = 5, value = 30)
time_slider = widgets.FloatSlider(min = 0.1, max = 2, step = 0.1, value = 1)

display(widgets.interactive(plot, K = K_slider, typ = fixed("lp"), S = fixed(40), 
                            time = time_slider, rf = rf_slider, sigma = sigma_slider))

### Short Call

In [ ]:
# Short Call
rf_slider = widgets.FloatSlider(min = 0.01, max = 0.2, step = 0.01, value = 0.1)
sigma_slider = widgets.FloatSlider(min = 0.1, max = 0.9, step = 0.1, value = 0.5)
K_slider = widgets.IntSlider(min = 10, max = 60, step = 5, value = 30)
time_slider = widgets.FloatSlider(min = 0.1, max = 2, step = 0.1, value = 1)

display(widgets.interactive(plot, K = K_slider, typ = fixed("sc"), S = fixed(40), 
                            time = time_slider, rf = rf_slider, sigma = sigma_slider))

### Short Put

In [ ]:
# Short Put
rf_slider = widgets.FloatSlider(min = 0.01, max = 0.2, step = 0.01, value = 0.1)
sigma_slider = widgets.FloatSlider(min = 0.1, max = 0.9, step = 0.1, value = 0.5)
K_slider = widgets.IntSlider(min = 10, max = 60, step = 5, value = 30)
time_slider = widgets.FloatSlider(min = 0.1, max = 2, step = 0.1, value = 1)

display(widgets.interactive(plot, K = K_slider, typ = fixed("sp"), S = fixed(40), 
                            time = time_slider, rf = rf_slider, sigma = sigma_slider))

Try out the sliders.
1. How does the price of a long call compare with its payoff?
2. How does the price of a long put compate with its payoff? Why do you think this is different?
3. If we were to plot the price of an American put, how do you think its price would compare to the payoff?
4. How do each of the 4 parameters (strike, volatility, risk-free rate, time to expiration) affect prices?

For the curious: https://aaronschlegel.me/black-scholes-formula-python.html

---

### Feedback

Please fill out the feedback form below regarding this lecture.

In [ ]:
feedback.ask()